# Хід роботи

In [56]:
from pycaret.datasets import get_data
data = get_data('traffic')

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,Rush Hour,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,1,5545
1,NaN,289.36,0.0,0.0,75,Clouds,0,4516
2,NaN,289.58,0.0,0.0,90,Clouds,0,4767
3,NaN,290.13,0.0,0.0,90,Clouds,0,5026
4,NaN,291.14,0.0,0.0,75,Clouds,0,4918


 Функція get_data('traffic') завантажує датасет "traffic" із вбудованої бібліотеки PyCaret і виводить перші 5 рядків для попереднього перегляду. Змінна data міститиме повний набір даних у форматі pandas DataFrame.


### Опис ознак датасету "traffic"
Для повного розуміння фізичного сенсу даних я дослідив датасет "traffic" у документації PyCaret та інших джерелах. Ось опис усіх ознак:
1. holiday: Описано нижче
2. temp: Температура повітря в градусах (ймовірно, за Цельсієм).
3. rain_1h: Кількість опадів (дощу) за останню годину в міліметрах.
4. snow_1h: Кількість снігу за останню годину в міліметрах.
5. clouds_all: Відсоток хмарності (0–100%).
6. weather_main: Описано нижче
7. Rush Hour: Бінарна ознака, що вказує на годину пік (1 — так, 0 — ні).
8. traffic_volume: Обсяг трафіку (цільова змінна), числове значення, що показує кількість транспортних засобів за певний період.


In [57]:
data.holiday.unique(), data.weather_main.unique() 

(array([nan, 'Columbus Day', 'Veterans Day', 'Thanksgiving Day',
        'Christmas Day', 'New Years Day', 'Washingtons Birthday',
        'Memorial Day', 'Independence Day', 'State Fair', 'Labor Day',
        'Martin Luther King Jr Day'], dtype=object),
 array(['Clouds', 'Clear', 'Rain', 'Drizzle', 'Mist', 'Haze', 'Fog',
        'Thunderstorm', 'Snow', 'Squall', 'Smoke'], dtype=object))

Ці ознаки дозволяють проаналізувати, як погодні умови, святкові дні та години пік впливають на обсяг трафіку.

### Вибір завдання для аналізу

Тепер визначаємо яке завдання машинного навчання є актуальним для цього датасету: регресія, класифікація чи кластеризація.

Цільова змінна ознака traffic_volume є числовою (наприклад, 5000 одиниць трафіку), що вказує на можливість використання регресії для передбачення точного значення обсягу трафіку на основі інших ознак. 

Оскільки traffic_volume є числовою ознакою, а мета — передбачити її значення, обираємо регресію як основне завдання.



## Аналіз даних та тренування моделей
### Налаштування експерименту

Параметр target='traffic_volume' вказує, що traffic_volume є цільовою змінною. Після виконання цієї команди PyCaret виведе таблицю з інформацією про датасет: кількість рядків, ознак, типи даних, пропущені значення тощо.

In [58]:
from pycaret import regression
regression.setup(data, target='traffic_volume')

,Description,Value
0,Session id,453
1,Target,traffic_volume
2,Target type,Regression
3,Original data shape,"(48204, 8)"
4,Transformed data shape,"(48204, 28)"
5,Transformed train set shape,"(33742, 28)"
6,Transformed test set shape,"(14462, 28)"
7,Numeric features,5
8,Categorical features,2
9,Rows with missing values,99.9%


## Порівняння моделей

Функція compare_models() тренує кілька регресійних моделей (лінійна регресія, Random Forest, XGBoost тощо) за допомогою крос-валідації та виводить таблицю з результатами. Найкраща модель (з найвищим R²) зберігається у змінній best_model.


In [59]:
best_model = regression.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1501.0133,3075721.9529,1753.6667,0.2175,0.9278,2.5599,0.0810
gbr,Gradient Boosting Regressor,1528.1412,3140943.0126,1772.1812,0.2009,0.9395,2.6188,0.1930
ada,AdaBoost Regressor,1576.2026,3276743.6409,1810.1130,0.1663,0.9713,2.7567,0.0860
knn,K Neighbors Regressor,1565.2794,3621327.7721,1902.8329,0.0787,0.9553,2.5488,0.0730
rf,Random Forest Regressor,1533.4523,3677551.1284,1917.5310,0.0644,0.9498,2.3651,0.6160
omp,Orthogonal Matching Pursuit,1707.9954,3830571.5112,1957.1187,0.0255,1.0188,2.9946,0.0220
dummy,Dummy Regressor,1741.7326,3932059.9539,1982.8692,-0.0003,1.0294,2.9304,0.0210
et,Extra Trees Regressor,1670.5193,4532201.1220,2128.7495,-0.1532,1.0434,2.5417,0.3870
dt,Decision Tree Regressor,1716.2383,5100029.6331,2258.0595,-0.2976,1.1107,2.4085,0.0290
huber,Huber Regressor,1614.7091,10728686.5607,2528.4214,-1.7344,0.9589,2.6998,0.1900


##  Оцінка продуктивності моделі

Для детального аналізу найкращої моделі використаємо функцію evaluate_model. Residuals Plot показує розподіл залишків (різницю між прогнозами та реальними значеннями). Prediction Error порівняння прогнозованих і фактичних значень. Feature Importance важливість кожної ознаки (наприклад, Rush Hour чи temp можуть мати найбільший вплив).



In [60]:
regression.evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

## Налаштування гіперпараметрів

Функція tune_model оптимізує гіперпараметри (наприклад, глибину дерева для Random Forest) за допомогою пошуку по сітці або випадкового пошуку. Після цього виводяться оновлені метрики.

In [61]:
tuned_model = regression.tune_model(best_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1516.4976,3104915.0723,1762.0769,0.2255,0.9453,4.1033
1,1495.9332,3059273.5837,1749.0779,0.2147,0.9011,1.6164
2,1487.8213,3025735.8015,1739.4642,0.2308,0.9322,2.3969
3,1516.0951,3147426.9117,1774.0989,0.1972,0.9107,1.4205
4,1502.0708,3067436.0977,1751.4097,0.2207,0.9392,2.0562
5,1549.2241,3236124.0587,1798.9230,0.2065,0.9616,3.1151
6,1492.3981,3064101.3474,1750.4575,0.1990,0.9237,3.0360
7,1490.4094,3069935.2951,1752.1231,0.2112,0.9356,2.1601
8,1497.5875,3017682.3624,1737.1478,0.2208,0.9009,1.3464


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


## Збереження моделі

Файл traffic_model.pkl міститиме повний пайплайн (обробку даних і модель).

In [62]:
regression.save_model(tuned_model, 'traffic_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['temp', 'rain_1h', 'snow_1h',
                                              'clouds_all', 'Rush Hour'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None,...
                  LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, impo

# Висновок

Ми завантажили датасет "traffic" із PyCaret та описали його ознаки: holiday, temp, rain_1h, snow_1h, clouds_all, weather_main, Rush Hour, traffic_volume. Основним завданням було передбачення traffic_volume, для чого ми обрали метод регресії.

У процесі аналізу ми налаштували експеримент, порівняли моделі, оцінили їхню продуктивність та здійснили налаштування гіперпараметрів. Після завершення роботи зберегли навчений алгоритм у файл traffic_model.pkl.

PyCaret значно спростив аналіз даних, дозволяючи виконати всі кроки з мінімальною кількістю коду. Метрики MAE, MSE, R², отримані на етапах порівняння та оцінки моделей, допомогли оцінити якість аналізу.